In [1]:
import pandas as pd
import sklearn.utils
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

In [2]:
data= pd.read_csv('Data_for_UCI_named.csv')
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [3]:
data= data.drop(columns= ['stab'], axis= 1)
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [4]:
X= data.iloc[:, : -1]
y= data['stabf']

In [5]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 1)

In [6]:
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [7]:
scaler= StandardScaler()
normalised_train_df= scaler.fit_transform(X_train)

In [8]:
normalised_train_df= pd.DataFrame(normalised_train_df, columns= X_train.columns)

In [9]:
normalised_train_df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


In [10]:
X_test= X_test.reset_index(drop= True)
normalised_test_df= scaler.transform(X_test)

In [11]:
normalised_test_df= pd.DataFrame(normalised_test_df, columns= X_test.columns)
normalised_test_df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.593951,-0.412733,1.503924,1.116943,0.403423,-1.492971,-0.785033,1.566781,-0.901007,1.167203,-1.507330,1.084726
1,0.202190,0.374416,-0.188800,-0.522268,-0.225967,-1.058483,0.420047,1.028627,-1.625721,-0.395660,1.414651,1.226011
2,-1.079044,-0.313745,-0.884634,0.017080,-0.943122,0.112653,0.801335,0.733004,1.457108,-1.438495,0.651821,-1.682168
3,-0.083120,-1.107327,0.372805,-1.708152,0.753990,-1.637972,0.403805,-0.088036,0.083322,-1.672322,-0.357714,1.055865
4,0.873921,1.438466,0.086662,1.715037,-0.153880,-0.007015,-0.197053,0.472315,0.136549,-1.469731,0.956396,-0.819727


In [12]:
rfc= RandomForestClassifier(random_state= 1)
rfc.fit(normalised_train_df, y_train)

C:\Users\ACER\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [13]:
new_predrfc= rfc.predict(normalised_test_df)

cm_rfc= confusion_matrix(y_true= y_test, y_pred= new_predrfc, labels= ['stable', 'unstable'])

cm_rfc

array([[ 627,   85],
       [ 134, 1154]], dtype=int64)

In [14]:
print(classification_report(y_test, new_predrfc, digits= 4))

              precision    recall  f1-score   support

      stable     0.8239    0.8806    0.8513       712
    unstable     0.9314    0.8960    0.9133      1288

    accuracy                         0.8905      2000
   macro avg     0.8777    0.8883    0.8823      2000
weighted avg     0.8931    0.8905    0.8913      2000



In [15]:
param= {'max_depth': 3, 'eta': 0.3, 'objective': 'multi:softprob',
       'num_class': 2}

steps= 20

In [16]:
xg_cla= xgb.XGBClassifier(param= param, random_state= 1)
xg_cla.fit(normalised_train_df, y_train)

C:\Users\ACER\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:00:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "param" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:00:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic',
              param={'eta': 0.3, 'max_depth': 3, 'num_class': 2,
                     'objective': 'multi:softprob'},
              random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [17]:
new_predxgb= xg_cla.predict(normalised_test_df)

cm_xgb= confusion_matrix(y_true= y_test, y_pred= new_predxgb, labels= ['stable', 'unstable'])
cm_xgb

array([[ 648,   64],
       [  45, 1243]], dtype=int64)

In [18]:
print(classification_report(y_test, new_predxgb, digits= 4))

              precision    recall  f1-score   support

      stable     0.9351    0.9101    0.9224       712
    unstable     0.9510    0.9651    0.9580      1288

    accuracy                         0.9455      2000
   macro avg     0.9430    0.9376    0.9402      2000
weighted avg     0.9453    0.9455    0.9453      2000



In [19]:
lig_cla= lgb.LGBMClassifier(random_state= 1)
lig_cla.fit(normalised_train_df, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
new_predlgb= lig_cla.predict(normalised_test_df)

cm_lig= confusion_matrix(y_true= y_test, y_pred= new_predlgb, labels= ['stable', 'unstable'])

cm_lig

array([[ 641,   71],
       [  50, 1238]], dtype=int64)

In [21]:
print(classification_report(y_test, new_predlgb, digits=4))

              precision    recall  f1-score   support

      stable     0.9276    0.9003    0.9138       712
    unstable     0.9458    0.9612    0.9534      1288

    accuracy                         0.9395      2000
   macro avg     0.9367    0.9307    0.9336      2000
weighted avg     0.9393    0.9395    0.9393      2000



In [22]:
etc= ExtraTreesClassifier(random_state= 1)
etc.fit(normalised_train_df, y_train)

C:\Users\ACER\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [23]:
new_predetc= etc.predict(normalised_test_df)

cm_etc= confusion_matrix(y_true= y_test, y_pred= new_predetc, labels= ['stable', 'unstable'])

cm_etc

array([[ 613,   99],
       [ 118, 1170]], dtype=int64)

In [24]:
print(classification_report(y_test, new_predetc, digits=4))

              precision    recall  f1-score   support

      stable     0.8386    0.8610    0.8496       712
    unstable     0.9220    0.9084    0.9151      1288

    accuracy                         0.8915      2000
   macro avg     0.8803    0.8847    0.8824      2000
weighted avg     0.8923    0.8915    0.8918      2000



In [25]:
etc_param= {'n_estimators': [50, 100, 300, 500, 1000],
           'min_samples_leaf':[1, 2, 4, 6, 8],
           'min_samples_split': [2, 3, 5, 7, 9],
           'max_features': ['auto', 'sqrt', 'log2', None]}

In [26]:
etc_rscv= RandomizedSearchCV(estimator= etc, param_distributions= etc_param,cv=5, n_iter=10, 
                             scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state= 1)

In [27]:
etc_rscv.fit(normalised_train_df, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=10, n_jobs=None,
                                                  oob_score=False,
          

In [28]:
etc_rscv.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [29]:
new_pred_rscv= etc_rscv.predict(normalised_test_df)

cm_etc_rscv= confusion_matrix(y_true= y_test, y_pred= new_pred_rscv, labels= ['stable', 'unstable'])

cm_etc_rscv

array([[ 619,   93],
       [  53, 1235]], dtype=int64)

In [30]:
print(classification_report(y_test, new_pred_rscv, digits= 4))

              precision    recall  f1-score   support

      stable     0.9211    0.8694    0.8945       712
    unstable     0.9300    0.9589    0.9442      1288

    accuracy                         0.9270      2000
   macro avg     0.9256    0.9141    0.9193      2000
weighted avg     0.9268    0.9270    0.9265      2000



In [31]:
print(etc.feature_importances_)

[0.11917257 0.11846717 0.10877427 0.11498868 0.04121199 0.03890201
 0.04128051 0.0417289  0.0886519  0.09533512 0.09713771 0.09434916]
